In [8]:
import scipy.io as io

import numpy as np
import robustsp as rsp
from scipy.optimize import least_squares as lsq

def arma_est_bip_mm(x,p,q):
    bip_s_est = rsp.arma_est_bip_s(x,p,q) #inno scale shouldnt be 0

    beta_hat_s = np.array([*bip_s_est['ar_coeffs'], *bip_s_est['ma_coeffs']])
    
    N = len(x)
    
    F_mm = lambda beta: 1/(N-p)*rsp.muler_rho2(rsp.arma_s_resid(x,beta,p,q)[0]) #/bip_s_est['inno_scale']
    
    F_bip_mm = lambda beta: 1/(N-p)*rsp.muler_rho2(rsp.bip_s_resid(x,beta,p,q)[0]) #/bip_s_est['inno_scale']

    beta_arma_mm = lsq(F_mm, -beta_hat_s,xtol=5*1e-7,ftol=5*1e-7,method='lm')['x']
    
    beta_bip_mm = lsq(F_bip_mm, -beta_hat_s,xtol=5*1e-7,ftol=5*1e-7,method='lm')['x']
    
    a = rsp.arma_s_resid(x, beta_arma_mm, p, q)[0]
    
    a_sc = rsp.m_scale(a) # innovations m-scale for ARMA model
    
    a_bip = rsp.bip_s_resid(x, beta_bip_mm, p, q)[0]
    
    a_bip_sc = rsp.m_scale(a_bip) # innovations m-scale for BIP-ARMA model
    
    # final parameter estimate uses the model that provides smallest m_scale
    beta_hat = beta_arma_mm if a_sc<a_bip_sc else beta_bip_mm
    
    # final m-scale
    a_m_sc = min(a_sc, a_bip_sc)
    
    # Output the results
    
    results = {'ar_coeffs': -beta_hat[:p],
               'ma_coeffs': -beta_hat[p:],
               'inno_scale': a_m_sc,
               'cleaned_signal': bip_s_est['cleaned_signal'],
               'ar_coeffs_init': bip_s_est['ar_coeffs'],
               'ma_coeffs_init': bip_s_est['ma_coeffs']}
    
    return results
    
x = io.loadmat('x_Ao')['x_ao'].flatten()
p=1
q=0
arma_est_bip_mm(x,1,0)

{'ar_coeffs': array([-0.80927153]),
 'ma_coeffs': array([], dtype=float64),
 'inno_scale': 1.8636460332736946,
 'cleaned_signal': array([-6.49013765e-01,  6.61955030e-01, -2.28889273e-01, -1.29272446e+00,
         1.82080111e+01, -4.97993051e+00, -1.01451233e+02, -1.59749207e+00,
        -1.47485510e+00, -1.61541228e+02, -2.33054509e+01, -2.60991402e-01,
        -1.71819785e+00, -4.98684129e-01, -3.70347069e+01,  4.13224263e+01,
        -2.24267954e+00, -3.06421502e+00,  4.31486261e+01,  1.00819958e+00,
         5.31402426e-01,  1.02878039e+00, -2.06018680e+01,  3.85707929e+00,
         6.24743109e+01, -7.46743742e-02, -4.77121547e-01, -1.88461787e+02,
        -5.51374952e-01, -7.33634961e-01,  6.83940465e-01,  6.13161714e-01,
         9.41819585e-01,  4.31245950e-01,  1.13340598e+00,  1.83546083e+00,
        -9.92229665e-01,  2.57926321e+00,  2.65410712e+00,  1.48749996e+00,
         1.79334658e+00, -8.16899028e+01,  5.64463052e-01,  1.06369281e+00,
         9.85611657e+01, -1.6681206